Kompetisi *Tabular Playground Series* bulan April 2021 ini cukup spesial karena terinspirasi oleh dataset [Titanic](https://www.kaggle.com/c/titanic/overview), di mana di dalamnya terdapat peubah numerikal maupun kategorikal yang lebih bermakna dan beraneka ragam, sangat cocok bagi pemula seperti saya untuk mempraktikkan keterampilan pemrosesan awal data. Mari, langsung kita garap kompetisi ini!

## 1. Memuat pustaka yang diperlukan

In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.ensemble import ExtraTreesClassifier, VotingClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score
import warnings
import os

warnings.simplefilter('ignore')

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## 2. Memuat berkas csv ke dalam dataframe pandas

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2021/train.csv',index_col = 'PassengerId')
test = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2021/test.csv',index_col = 'PassengerId')
submission = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2021/sample_submission.csv')

Secara sekilas, berikut adalah isi dari dataframe train dan test. Bagi yang pernah mengikuti kompetisi Titanic sebelumnya pasti tidak merasa asing dengan peubah (*feature*) yang ada, seperti SibSp (*siblings and spouse*), Parch (*parents and children*), dll. Silakan merujuk pada kompetisi tersebut untuk penjelasan lebih lengkap dari masing-masing peubah (tautan di bagian awal notebook). Ini mungkin kemajuan yang perlu dicatat untuk kompetisi ini karena peubah yang disajikan semakin beragam. Keberagaman peubah yang ada membuat saya merasa diajak untuk menyadari bahwa data tidaklah semata-mata angka, melainkan juga memiliki narasi. Narasilah yang membuat hubungan sebab-akibat yang ada terasa lebih masuk akal.

In [ ]:
train.head()

In [ ]:
test.head()

## 3. Memeriksa keberadaan *missing values*

Seperti biasa, saya memeriksa terlebih dahulu apakah ada data ganda (duplikat), baik pada dataset train maupun test, yaitu dengan menjalankan kode `train.duplicated().any()` dan `test.duplicated().any()`, dan ternyata kedua dataset tidak mengandung data ganda. Seandainya ada, adalah praktik yang baik untuk membuang data ganda tersebut.

Kemudian, termasuk tahapan baku juga untuk memeriksa keberadaan *missing values* pada kedua dataset, sebagaimana ditunjukkan di bawah ini. Keberadaan *missing values* dihitung pada setiap peubah atau kolom sebagai persentase terhadap jumlah keseluruhan observasi. Mirip dengan dataset Titanic yang asli, peubah Fare, Embarked, Age, Ticket, dan Cabin berturut-turut memiliki *missing values* dengan proporsi dari yang paling sedikit ke yang paling banyak.

In [ ]:
train.isna().sum()/train.shape[0] * 100

In [ ]:
test.isna().sum()/test.shape[0] * 100

Secara visual, proporsi `missing values` dapat diamati pada grafik *heatmap* berikut.

In [ ]:
sns.heatmap(train.isna(),yticklabels = False,cbar = False,cmap = 'viridis')

In [ ]:
sns.heatmap(test.isna(),yticklabels = False,cbar = False,cmap = 'viridis')

## 3. Imputasi *missing values* dan pemrosesan data awal

Terhadap *missing values* yang sudah ditemukan sebelumnya, sebagian diimputasi dengan nilai rata-rata dan sebagian diisi dengan string `Missing`.

Pada tahap ini juga sekaligus dilakukan *binning* pada beberapa peubah. Misalnya peubah Age di-*binning* ke dalam kategori berikut: 80s, 70s, 60s, ..., 0s menggunakan metode apply() dan fungsi lambda.

In [ ]:
train['Age'].fillna(train['Age'].mean(),inplace = True)
train['Age'] = train['Age'].apply(lambda x: '80s' if x >= 80 else '70s' if x>=70 else '60s' if x>=60 else '50s' if x>=50 else '40s' if x>=40 else '30s' if x>=30 else '20s' if x>=20 else '10s' if x>=10 else '0s')
test['Age'].fillna(test['Age'].mean(),inplace = True)
test['Age'] = test['Age'].apply(lambda x: '80s' if x >= 80 else '70s' if x>=70 else '60s' if x>=60 else '50s' if x>=50 else '40s' if x>=40 else '30s' if x>=30 else '20s' if x>=20 else '10s' if x>=10 else '0s')

Untuk peubah Name, terdapat sedikit perbedaan dengan dataset Titanic yang asli, di mana tidak terdapatnya gelar atau sebutan seperti *Sir*, *Lady*, dst. Biasanya informasi tentang gelar bisa dianggap menandakan kelas sosial dan kemungkinan berhubungan dengan keistimewaan seseorang untuk diselamatkan terlebih dahulu. Namun demikian, walaupun tidak ada informasi gelar tadi, saya mencoba untuk menarik informasi marga atau nama keluarga dari setiap penumpang. Marga seperti Smith adalah marga yang sangat banyak dijumpai di antara penumpang. Ada juga marga-marga langka seperti Barefield, Proffer, dan sejenisnya. Terhadap marga tersebut, saya melakukan pengelompokan berdasarkan keumuman atau kelangkaan sebagai berikut.

In [ ]:
train['FamName'] = train['Name'].str.extract('([A-Za-z]+)\,', expand = False)
test['FamName'] = test['Name'].str.extract('([A-Za-z]+)\,', expand = False)
FamName = train['FamName'].append(test['FamName']).value_counts()
FamName = FamName.apply(lambda x: 'UltraCommon' if x >= 512 else 'VeryCommon' if x >= 256 else 'ModeratelyCommon' if x >= 128 else 'Common' if x >= 64 else 'SlightlyCommon' if x >= 32 else 'SlightlyRare' if x >= 16 else 'Rare' if x >= 8 else 'ModeratelyRare' if x >= 4 else 'VeryRare' if x >= 2 else 'UltraRare')
train['FamName'] = train['FamName'].apply(lambda x: FamName[x])
test['FamName'] = test['FamName'].apply(lambda x: FamName[x])
train.drop(columns = 'Name',inplace = True)
test.drop(columns = 'Name',inplace = True)

Untuk peubah Ticket, dapat diamati bahwa terdapat pola khusus, misalnya tiket yang berupa alfanumerikal atau numerikal. Dalam hal ini, saya melakukan pengelompokan berdasarkan pola tersebut, khususnya mengambil karakter pertama dari setiap nomor tiket. Secara naif, anggaplah bahwa pola tiket tertentu mencerminkan fasilitas yang dapat dinikmati oleh penumpang, yang bisa saja terkait dengan peluang keselamatan penumpang tersebut. Kemudian, khusus untuk *missing values*, saya mengisinya dengan string `Missing`.

In [ ]:
train['Ticket'] = train['Ticket'].apply(lambda x: x[0] if type(x) == str else 'Missing')
test['Ticket'] = test['Ticket'].apply(lambda x: x[0] if type(x) == str else 'Missing')

Hal serupa di atas juga saya terapkan untuk mengkategorisasikan peubah Cabin. Patut diduga bahwa huruf pertama pada peubah Cabin mencerminkan nomor geladak. Di sisi lain, jika asumsi tersebut benar, nomor geladak juga dapat dianggap memiliki kontribusi terhadap peluang selamatnya seorang penumpang. Sebagaimana yang sudah saya tulis di [notebook](https://www.kaggle.com/bagusbpg/my-3rd-notebook) saya untuk kompetisi Titanic dan juga berdasarkan keterangan pada [laman](https://en.wikipedia.org/wiki/Titanic) Wikipedia, geladak tertentu memiliki akses yang lebih mudah ke kapal sekoci. Untuk *missing values*, saya mengisinya dengan string`Missing`, sama seperti pada langkah sebelumnya.

In [ ]:
train['Cabin'] = train['Cabin'].apply(lambda x: x[0] if type(x) == str else 'Missing')
test['Cabin'] = test['Cabin'].apply(lambda x: x[0] if type(x) == str else 'Missing')

Kemudian, dari peubah SibSp dan Parch, saya bisa membuat peubah baru FamSize. Langkah ini sudah menjadi kelaziman yang dapat dijumpai pada mungkin hampir seluruh *notebook* pada kompetisi Titanic yang asli. Hal khusus yang saya lakukan kali ini adalah melakukan *binning* terhadap peubah FamSize berdasarkan ukuran keluarga, mulai dari kategori Alone, Couple, Small, dst. Bisa jadi, keluarga dengan jumlah anggota tertentu memiliki peluang selamat yang lebih tinggi daripada yang lain.

In [ ]:
train['FamSize'] = train['SibSp'] + train['Parch'] + 1
train['FamSize'] = train['FamSize'].apply(lambda x: 'VeryBig' if x >= 12 else 'Big' if x >= 8 else 'Medium' if x >= 5 else 'Small' if x >= 3 else 'Couple' if x ==2 else 'Alone')
test['FamSize'] = test['SibSp'] + test['Parch'] + 1
test['FamSize'] = test['FamSize'].apply(lambda x: 'VeryBig' if x >= 12 else 'Big' if x >= 8 else 'Medium' if x >= 5 else 'Small' if x >= 3 else 'Couple' if x ==2 else 'Alone')

Untuk peubah Fare, strateginya juga kira-kira mirip dengan yang dilakukan untuk peubah Age. Saya melakukan *binning* terhadap peubah Fare ini untuk menghasilkan kategori seperti Rich, Poor, dst. Bisa jadi harga tiket tertentu adalah tanda seorang penumpang memiliki keistimewaan, termasuk dalam hal akses terhadap kapal sekoci.

In [ ]:
train['Fare'].fillna(train['Fare'].mean(),inplace = True)
train['Fare'] = train['Fare'].apply(lambda x: 'CrazyRich' if x >= 640 else 'UltraRich' if x >= 320 else 'VeryRich' if x >= 160 else 'Rich' if x >= 80 else 'SlightlyRich' if x >= 40 else 'SlightlyPoor' if x >= 20 else 'Poor' if x >= 10 else 'VeryPoor' if x >= 5 else 'UltraPoor')
test['Fare'].fillna(test['Fare'].mean(),inplace = True)
test['Fare'] = test['Fare'].apply(lambda x: 'CrazyRich' if x >= 640 else 'UltraRich' if x >= 320 else 'VeryRich' if x >= 160 else 'Rich' if x >= 80 else 'SlightlyRich' if x >= 40 else 'SlightlyPoor' if x >= 20 else 'Poor' if x >= 10 else 'VeryPoor' if x >= 5 else 'UltraPoor')

Sampai dengan tahap ini, hasil imputasi *missing values* dan *binning* adalah sebagai berikut.

Catatan: peubah Embarked masih menyisakan missing values, dan saya akan membiarkannya seperti itu.

In [ ]:
train.head()

In [ ]:
test.head()

Untuk peubah kategorikal yang dihasilkan, saya mencoba menerapkan skema one hot encoding melalui *method* get_dummies()

In [ ]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

Setelah semua peubah sudah dipastikan hanya mengandung isian numerikal, saya melakukan pemisahan dataset training menjadi masukan X dan luaran y.

In [ ]:
X = train.drop(columns = 'Survived')
y = train['Survived']

Hasilnya, masukan X, luaran y, dan sekaligus dataset test, adalah sebagai berikut.

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
test.head()

## 4. Pemodelan dan penyetelan parameter

Pada kompetisi ini, saya mencoba melakukan penyetelan (*tuning*) dan menerapkan klasifikasi berdasar voting untuk pertama kalinya.

Untuk penyetelan, saya tidak memakai *method* yang lebih lazim seperti GridSearchCV() -- mungkin ke depan akan saya coba. Kali ini saya coba mengadopsi cara yang diajarkan oleh Mike Bernico pada salah satu [video](https://www.youtube.com/watch?v=0GrciaGYzV0) di kanal youtube-nya.

Pertama, saya menentukan parameter n_estimators paling optimal pada ExtraTreesClassifier sebagai berikut. Di susul kemudian, saya berturut-turut menjalankan cara yang sama untuk memperoleh nilai paling optimal untuk parameter max_depth, min_samples_split, dan min_samples_leaf.

In [ ]:
results = []
score = 0
n_estimators = [100,200,500,1000]

for trees in n_estimators:
    clf = ExtraTreesClassifier(n_estimators = trees,oob_score = True,bootstrap = True,n_jobs = -1,random_state = 42)
    clf.fit(X,y)
    if clf.oob_score_ > score:
        score = clf.oob_score_
        best = trees
    results.append(clf.oob_score_)

print(f'n_estimators = {best}')
pd.Series(results,n_estimators).plot()

In [ ]:
results = []
score = 0
max_depth = [13,15, 17, 18, 19]

for depth in max_depth:
    clf = ExtraTreesClassifier(n_estimators = 1000,max_depth = depth,oob_score = True,bootstrap = True,n_jobs = -1,random_state = 42)
    clf.fit(X,y)
    if clf.oob_score_ > score:
        score = clf.oob_score_
        best = depth
    results.append(clf.oob_score_)

print(f'max_depth = {best}')
pd.Series(results,max_depth).plot()

In [ ]:
results = []
score
min_samples_split = [24,25,26,27]

for split in min_samples_split:
    clf = ExtraTreesClassifier(n_estimators = 1000,max_depth = 17,min_samples_split = split,oob_score = True,bootstrap = True,n_jobs = -1,random_state = 42)
    clf.fit(X,y)
    if clf.oob_score_ > score:
        score = clf.oob_score_
        best = split
    results.append(clf.oob_score_)

print(f'min_samples_split = {best}')
pd.Series(results,min_samples_split).plot()

In [ ]:
results = []
score = 0
min_samples_leaf = [14,17,18,19,20]

for leaves in min_samples_leaf:
    clf = ExtraTreesClassifier(n_estimators = 1000,max_depth = 17,min_samples_split = 25,min_samples_leaf = leaves,oob_score = True,bootstrap = True,n_jobs = -1,random_state = 42)
    clf.fit(X,y)
    if clf.oob_score_ > score:
        score = clf.oob_score_
        best = leaves
    results.append(clf.oob_score_)

print(f'min_samples_leaf = {best}')
pd.Series(results,min_samples_leaf).plot()

In [ ]:
clf_ext = ExtraTreesClassifier(n_estimators = 1000,max_depth = 17,min_samples_split = 25,min_samples_leaf = 18,n_jobs = -1,random_state = 42)

ExtraTreesClassifier adalah estimator *ensemble* yang menerapkan konsep *bagging*. Di sini saya ingin mengkombinasikan estimator *bagging* dengan estimator lain yang menerapkan konsep *boosting*, yaitu LGBMClassifier. Untuk LGBMClassifier ini, saya juga melakukan pencarian nilai optimal untuk beberapa parameter sebagaimana yang saya lakukan sebelumnya pada estimator ExtraTreesClassifier. Hasil akhirnya adalah sebagai berikut

In [ ]:
clf_lgbm = LGBMClassifier(boosting_type = 'dart',num_leaves = 32,max_depth = 10,colsample_bytree = 0.8,extra_trees = True,n_jobs = -1,random_state = 42)

Setelah penyetelan selesai, kedua estimator dikombinasikan melalui suatu estimator final yang menerapkan *soft voting*. Saya beranggapan bahwa estimator *bagging* dan *boosting* akan saling melengkapi satu sama lain, sesuai dengan keunggulan dan kelemahan masing-masing, dengan harapan akan dihasilkan estimator final yang memiliki kekuatan prediktif yang lebih mumpuni.

In [ ]:
clf = VotingClassifier(estimators=[('ext',clf_ext),('lgbm', clf_lgbm)], voting='soft')
clf = clf.fit(X,y)

## 5. Prediksi

Ini adalah tahap akhir dari keseluruhan proses, yaitu dilakukannya prediksi terhadap dataset test menggunakan model yang sudah di-*fit*-kan terhadap masukan X dan luaran y. Skor akurasi yang dihasilkan dapat dilihat pada *public leaderboard*.

In [ ]:
submission['Survived'] = pd.Series(clf.predict(test))
submission.set_index('PassengerId').to_csv('submission.csv')